# KOPPU: System Pulse Check

This notebook serves as a basic sanity check for the KOPPU system. It instantiates an OPU, connects the Brian2 driver, and runs a simple simulation to verify that the digital twin is responsive and producing neural activity.

In [ ]:
import pykoppu as pk
import numpy as np

# 1. Setup OPU & Driver
print("Initializing OPU...")
opu = pk.opu.OPU(capacity=50)
driver = pk.electrophysiology.connect("cpu", opu)

# 2. Define a Simple Program (Pulse Check)
# We will inject some noise and run for a short duration.
from pykoppu.biocompiler.isa import Instruction, OpCode

program = [
    Instruction(OpCode.ALC, [50]),       # Allocate 50 neurons
    Instruction(OpCode.SIG, [2.0]),      # Set noise level to 2.0 mV
    Instruction(OpCode.RUN, [0.5]),      # Run for 500ms
    Instruction(OpCode.RD, [])           # Read state
]

# 3. Execute Manually
print("Executing Pulse Check...")
driver.connect()
try:
    # Execute returns (state, energy, spikes)
    final_state, energy_trace, spike_data = driver.execute(program)
finally:
    driver.disconnect()

# 4. Visualize Results
print("System Pulse Check Complete.")

# Wrap in SimulationResult for visualization
from pykoppu.oos.result import SimulationResult

result = SimulationResult(
    solution=final_state,
    energy_history=energy_trace,
    spikes=spike_data,
    metadata={"description": "System Pulse Check"}
)

result.plot()